# I MODIFIED THIS AND NOW IT DOESN'T WORK

In [21]:
import numpy as np
import tensorflow.compat.v1 as tf
import pyspiel
from open_spiel.python.games import block_dominoes
from open_spiel.python import rl_environment
from open_spiel.python.pytorch import dqn as dqn_pt
from open_spiel.python.algorithms import dqn
from open_spiel.python.algorithms import random_agent

In [22]:
games_list = pyspiel.registered_names()

print("Registered games:")
print(games_list)

Registered games:
['2048', 'add_noise', 'amazons', 'backgammon', 'bargaining', 'battleship', 'blackjack', 'blotto', 'breakthrough', 'bridge', 'bridge_uncontested_bidding', 'catch', 'chat_game', 'checkers', 'chess', 'cliff_walking', 'clobber', 'coin_game', 'colored_trails', 'connect_four', 'coop_box_pushing', 'coop_to_1p', 'coordinated_mp', 'crazy_eights', 'cursor_go', 'dark_chess', 'dark_hex', 'dark_hex_ir', 'deep_sea', 'dots_and_boxes', 'dou_dizhu', 'efg_game', 'euchre', 'first_sealed_auction', 'gin_rummy', 'go', 'goofspiel', 'havannah', 'hearts', 'hex', 'kriegspiel', 'kuhn_poker', 'laser_tag', 'leduc_poker', 'lewis_signaling', 'liars_dice', 'liars_dice_ir', 'maedn', 'mancala', 'markov_soccer', 'matching_pennies_3p', 'matrix_bos', 'matrix_brps', 'matrix_cd', 'matrix_coordination', 'matrix_mp', 'matrix_pd', 'matrix_rps', 'matrix_rpsw', 'matrix_sh', 'matrix_shapleys_game', 'mfg_crowd_modelling', 'mfg_crowd_modelling_2d', 'mfg_dynamic_routing', 'mfg_garnet', 'misere', 'morpion_solitaire'

In [23]:
def eval_against_random_bots(env, trained_agents, random_agents, num_episodes):
  """Evaluates `trained_agents` against `random_agents` for `num_episodes`."""
  num_players = len(trained_agents)
  sum_episode_rewards = np.zeros(num_players)
  for player_pos in range(num_players):
    cur_agents = random_agents[:]
    cur_agents[player_pos] = trained_agents[player_pos]
    for _ in range(num_episodes):
      time_step = env.reset()
      episode_rewards = 0
      while not time_step.last():
        player_id = time_step.observations["current_player"]
        if env.is_turn_based:
          agent_output = cur_agents[player_id].step(
              time_step, is_evaluation=True)
          action_list = [agent_output.action]
        else:
          agents_output = [
              agent.step(time_step, is_evaluation=True) for agent in cur_agents
          ]
          action_list = [agent_output.action for agent_output in agents_output]
        time_step = env.step(action_list)
        episode_rewards += time_step.rewards[player_pos]
      sum_episode_rewards[player_pos] += episode_rewards
  return sum_episode_rewards / num_episodes

In [24]:
def pt_main(game,
            # config,
            checkpoint_dir,
            num_train_episodes,
            eval_every,
            hidden_layers_sizes,
            replay_buffer_capacity,
            batch_size):
  num_players = 2

  env = rl_environment.Environment(game) # rl_environment.Environment(game, **config)
  info_state_size = env.observation_spec()["info_state"][0]
  num_actions = env.action_spec()["num_actions"]

  # random agents for evaluation
  random_agents = [
      random_agent.RandomAgent(player_id=idx, num_actions=num_actions)
      for idx in range(num_players)
  ]


  hidden_layers_sizes = [int(l) for l in hidden_layers_sizes]
  # pylint: disable=g-complex-comprehension
  agents = [
      dqn_pt.DQN(
          player_id=idx,
          state_representation_size=info_state_size,
          num_actions=num_actions,
          hidden_layers_sizes=hidden_layers_sizes,
          replay_buffer_capacity=replay_buffer_capacity,
          batch_size=batch_size) for idx in range(num_players)
  ]
  result = []
  for ep in range(num_train_episodes):
    if (ep + 1) % eval_every == 0:
      r_mean = eval_against_random_bots(env, agents, random_agents, 1000)
      result.append(r_mean)
      print("[%s] Mean episode rewards %s" %(ep + 1, r_mean))

    time_step = env.reset()
    while not time_step.last():
      player_id = time_step.observations["current_player"]
      if env.is_turn_based:
        agent_output = agents[player_id].step(time_step)
        action_list = [agent_output.action]
      else:
        agents_output = [agent.step(time_step) for agent in agents]
        action_list = [agent_output.action for agent_output in agents_output]
      time_step = env.step(action_list)

    # Episode is over, step all agents with final info state.
    for agent in agents:
      agent.step(time_step)
  return result

In [25]:
def tf_main(game,
            # config,
            checkpoint_dir,
            num_train_episodes,
            eval_every,
            hidden_layers_sizes,
            replay_buffer_capacity,batch_size):
  num_players = 2

  env = rl_environment.Environment(game)# rl_environment.Environment(game, **config)
  info_state_size = env.observation_spec()["info_state"][0]
  num_actions = env.action_spec()["num_actions"]

  # random agents for evaluation
  random_agents = [
      random_agent.RandomAgent(player_id=idx, num_actions=num_actions)
      for idx in range(num_players)
  ]

  with tf.Session() as sess:
    hidden_layers_sizes = [int(l) for l in hidden_layers_sizes]
    # pylint: disable=g-complex-comprehension
    agents = [
        dqn.DQN(
            session=sess,
            player_id=idx,
            state_representation_size=info_state_size,
            num_actions=num_actions,
            hidden_layers_sizes=hidden_layers_sizes,
            replay_buffer_capacity=replay_buffer_capacity,
            batch_size=batch_size) for idx in range(num_players)
    ]
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    
    result_tf = []
    for ep in range(num_train_episodes):
      if (ep + 1) % eval_every == 0:
        r_mean = eval_against_random_bots(env, agents, random_agents, 1000)
        result_tf.append(r_mean)
        print("[%s] Mean episode rewards %s" %(ep + 1, r_mean))

      time_step = env.reset()
      while not time_step.last():
        player_id = time_step.observations["current_player"]
        if env.is_turn_based:
          agent_output = agents[player_id].step(time_step)
          action_list = [agent_output.action]
        else:
          agents_output = [agent.step(time_step) for agent in agents]
          action_list = [agent_output.action for agent_output in agents_output]
        time_step = env.step(action_list)

      # Episode is over, step all agents with final info state.
      for agent in agents:
        agent.step(time_step)
  return result_tf

In [26]:
checkpoint_dir = "/tmp/dqn_test"
num_train_episodes = 10000
eval_every = 100

hidden_layers_sizes = [64, 64]
replay_buffer_capacity = int(1e5)
batch_size = 32

# BREAKTHROUGH

In [27]:
#game = "breakthrough"
# config = {"columns": 5, "rows": 5}
game = 'python_block_dominoes'

In [28]:
pt_result = pt_main(game,
                    # config,
                    checkpoint_dir,
                    num_train_episodes,
                    eval_every,
                    hidden_layers_sizes,
                    replay_buffer_capacity,batch_size)

[100] Mean episode rewards [ 3.648 -1.1  ]


RuntimeError: masked_fill_ only supports boolean masks, but got mask with dtype float

In [29]:
import matplotlib.pyplot as plt

ep = [x for x in range(len(pt_result))]
pt_r_mean0 = [y[0] for y in pt_result]
pt_r_mean1 = [y[1] for y in pt_result]

plt.plot(ep,pt_r_mean0, c='red')
plt.plot(ep,pt_r_mean1, c='blue')
plt.xlabel('Episode')
plt.ylabel('Mean episode rewards')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [30]:
result_tf = tf_main(game,
                    # config,
                    checkpoint_dir,
                    num_train_episodes,
                    eval_every,
                    hidden_layers_sizes,
                    replay_buffer_capacity,batch_size)

2024-04-10 11:06:59.135850: W tensorflow/c/c_api.cc:305] Operation '{name:'mlp_3/bias_2/Assign' id:458 op device:{requested: '', assigned: ''} def:{{{node mlp_3/bias_2/Assign}} = Assign[T=DT_FLOAT, _class=["loc:@mlp_3/bias_2"], _has_manual_control_dependencies=true, use_locking=true, validate_shape=true](mlp_3/bias_2, mlp_3/zeros_2)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


[100] Mean episode rewards [ 2.995 -3.653]
[200] Mean episode rewards [ 3.55  -2.823]
[300] Mean episode rewards [ 4.374 -3.707]
[400] Mean episode rewards [ 4.252 -3.171]
[500] Mean episode rewards [ 4.304 -2.263]
[600] Mean episode rewards [ 5.56  -3.963]
[700] Mean episode rewards [ 5.596 -2.491]
[800] Mean episode rewards [ 5.739 -2.559]
[900] Mean episode rewards [ 4.884 -3.927]
[1000] Mean episode rewards [ 4.238 -2.987]
[1100] Mean episode rewards [ 3.849 -2.482]
[1200] Mean episode rewards [ 5.432 -1.558]
[1300] Mean episode rewards [ 4.324 -2.152]
[1400] Mean episode rewards [ 6.353 -2.16 ]
[1500] Mean episode rewards [ 5.341 -2.335]
[1600] Mean episode rewards [ 5.71  -2.491]
[1700] Mean episode rewards [ 5.051 -2.521]
[1800] Mean episode rewards [ 5.021 -2.838]
[1900] Mean episode rewards [ 6.509 -2.764]
[2000] Mean episode rewards [ 6.3   -2.247]
[2100] Mean episode rewards [ 6.246 -1.54 ]
[2200] Mean episode rewards [ 6.062 -1.967]
[2300] Mean episode rewards [ 5.728 -1.25

In [8]:
ep = [x for x in range(len(result_tf))]
tf_r_mean0 = [y[0] for y in result_tf]
tf_r_mean1 = [y[1] for y in result_tf]

plt.plot(ep,tf_r_mean0, c='red')
plt.plot(ep,tf_r_mean1, c='blue')
plt.xlabel('Episode')
plt.ylabel('Mean episode rewards')
plt.show()

NameError: name 'result_tf' is not defined

In [ ]:
plt.plot(ep,pt_r_mean0, c='skyblue')
plt.plot(ep,pt_r_mean1, c='skyblue', linestyle='dashed')
plt.plot(ep,tf_r_mean0, c='pink')
plt.plot(ep,tf_r_mean1, c='pink', linestyle='dashed')
plt.xlabel('Episode')
plt.ylabel('Mean episode rewards')
plt.show()

# TIC-TAC-TOE

In [9]:
game = "tic_tac_toe"
config = {}
num_train_episodes = 20000
eval_every = 1000

In [10]:
pt_result = pt_main(game,
                    config,
                    checkpoint_dir,
                    num_train_episodes,
                    eval_every,
                    hidden_layers_sizes,
                    replay_buffer_capacity,batch_size)

[1000] Mean episode rewards [0.823 0.112]
[2000] Mean episode rewards [0.769 0.089]
[3000] Mean episode rewards [0.883 0.161]
[4000] Mean episode rewards [0.723 0.229]
[5000] Mean episode rewards [0.424 0.125]
[6000] Mean episode rewards [0.54  0.246]
[7000] Mean episode rewards [0.637 0.244]
[8000] Mean episode rewards [0.794 0.236]
[9000] Mean episode rewards [0.643 0.148]
[10000] Mean episode rewards [0.813 0.148]
[11000] Mean episode rewards [0.626 0.17 ]
[12000] Mean episode rewards [0.622 0.188]
[13000] Mean episode rewards [0.874 0.244]
[14000] Mean episode rewards [0.856 0.183]
[15000] Mean episode rewards [0.825 0.28 ]
[16000] Mean episode rewards [0.82  0.361]
[17000] Mean episode rewards [0.847 0.241]
[18000] Mean episode rewards [0.869 0.296]
[19000] Mean episode rewards [0.904 0.261]
[20000] Mean episode rewards [0.858 0.277]


In [ ]:
import matplotlib.pyplot as plt

ep = [x for x in range(len(pt_result))]
pt_r_mean0 = [y[0] for y in pt_result]
pt_r_mean1 = [y[1] for y in pt_result]

plt.plot(ep,pt_r_mean0, c='red')
plt.plot(ep,pt_r_mean1, c='blue')
plt.xlabel('Episode')
plt.ylabel('Mean episode rewards')
plt.show()

In [12]:
result_tf = tf_main(game,
                    config,
                    checkpoint_dir,
                    num_train_episodes,
                    eval_every,
                    hidden_layers_sizes,
                    replay_buffer_capacity,batch_size)

[1000] Mean episode rewards [ 0.493 -0.204]
[2000] Mean episode rewards [ 0.346 -0.244]
[3000] Mean episode rewards [ 0.537 -0.054]
[4000] Mean episode rewards [ 0.464 -0.059]
[5000] Mean episode rewards [0.46  0.144]
[6000] Mean episode rewards [0.442 0.08 ]
[7000] Mean episode rewards [0.606 0.068]
[8000] Mean episode rewards [0.447 0.165]
[9000] Mean episode rewards [0.702 0.196]
[10000] Mean episode rewards [0.694 0.227]
[11000] Mean episode rewards [0.757 0.213]
[12000] Mean episode rewards [0.829 0.149]
[13000] Mean episode rewards [0.733 0.186]
[14000] Mean episode rewards [0.8   0.318]
[15000] Mean episode rewards [0.849 0.308]
[16000] Mean episode rewards [0.789 0.198]
[17000] Mean episode rewards [0.825 0.353]
[18000] Mean episode rewards [0.804 0.367]
[19000] Mean episode rewards [0.827 0.355]
[20000] Mean episode rewards [0.796 0.368]


In [ ]:
ep = [x for x in range(len(result_tf))]
tf_r_mean0 = [y[0] for y in result_tf]
tf_r_mean1 = [y[1] for y in result_tf]

plt.plot(ep,tf_r_mean0, c='red')
plt.plot(ep,tf_r_mean1, c='blue')
plt.xlabel('Episode')
plt.ylabel('Mean episode rewards')
plt.show()

In [ ]:
plt.plot(ep,pt_r_mean0, c='skyblue')
plt.plot(ep,pt_r_mean1, c='skyblue', linestyle='dashed')
plt.plot(ep,tf_r_mean0, c='pink')
plt.plot(ep,tf_r_mean1, c='pink', linestyle='dashed')
plt.xlabel('Episode')
plt.ylabel('Mean episode rewards')
plt.show()